In [1]:
# pip install https://github.com/acpopescu/bitsandbytes/releases/download/v0.37.2-win.1/bitsandbytes-0.37.2-py3-none-any.whl
# pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117
# pip install bitsandbytes-windows
# pip install torch transformers accelerate

In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import os

# torch.cuda.is_available()
# torch.cuda.device_count()
# torch.cuda.current_device()
# torch.cuda.get_device_name(0)
# torch.cuda.get_device_name(1)

# use an A100 on Colab Pro (or any system with >30GB VRAM on your own machine) to load this in bf16
# if not available, use a GPU with minimum 20GB VRAM to load this in 8bit, or with minimum 12GB of VRAM to load in 4bit
# on Colab, works with a V100 but crashes on a T4

# downloading the model and then loading it to memory step takes around 10 minutes the first time. Be patient :)

torch.cuda.empty_cache()

c:\Users\cemticsai\anaconda3\envs\torchgpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import psutil

def clear_ram_cache():
    psutil.virtual_memory().available  # This can help free up some cached memory

# Call the function to attempt to clear RAM cache
clear_ram_cache()

In [4]:
#Import data
df = pd.read_excel(r"C:\Users\cemticsai\Documents\Projects\text2sql\SQL code\SQL data\demo data\clutser_data.xlsx")

In [5]:
df.shape

(355467, 28)

In [6]:
df.columns

Index(['customer', 'sitename', 'segment', 'churn_tag', 'voip_calls_count',
       'total_data_volume_mb', 'total_revenue', 'voice_calls_count',
       'segment_churn_tag', 'voip_consitant_tag', 'work_pattern',
       'voip1_user_tag', 'ppsf_tag', 'work_from_home_tag', 'data_user_tag',
       'voip2_user_tag', 'wifi_user_tag_new', 'y_pred',
       'per_difference_outgoing_calls', 'per_difference_fb_act_hour',
       'outgoingcallsdormancydays', 'per_difference_data_volume',
       'recharge_recency', 'bonus_frequency_last_3_months',
       'paidvideouserstag', 'news_financial_rider_users_tag', 'throughput',
       'video_user'],
      dtype='object')

In [7]:
# Import data and load it to Database

import sqlite3

# Connect to the database (or create it if it doesn't exist)
conn = sqlite3.connect('democustomerdata.sqlite')
c = conn.cursor()

# Create the table
c.execute('''CREATE TABLE democustomerdata (
    customer VARCHAR(5) NOT NULL,
    sitename VARCHAR(10) NOT NULL,
    segment VARCHAR(2) CHECK (segment IN ('MV', 'HV')) ,
    churn_tag VARCHAR(20) CHECK(churn_tag IN("Degrowth","Consistent")),
    voip_calls_count FLOAT,
    total_data_volume_mb FLOAT,
    total_revenue FLOAT,
    voice_calls_count FLOAT,
    segment_churn_tag VARCHAR(30) CHECK(segment_churn_tag IN("HV_Consistent_New","MV_Degrowth_New","MV_Consistent_New","HV_Degrowth_New")),
    voip_consistent_tag VARCHAR(30)CHECK (voip_consistent_tag IN("Zero","Consitent  users","in Consitent  users","recent")),
    work_pattern VARCHAR(30)CHECK( work_pattern IN("Inconsistent Worker","Consistent Workers","Home Bound","Others")),
    voip1_user_tag VARCHAR(20),
    ppsf_tag VARCHAR(10)CHECK(ppsf_tag IN("Low","Medium","High","Very High","NA")),
    work_from_home_tag VARCHAR(15) CHECK(work_from_home_tag IN ("Work from Office","Home bound","Work from Home","Other")),
    data_user_tag VARCHAR(10) CHECK(data_user_tag IN("very_high","high","low","very_low","mid")),
    voip2_user_tag VARCHAR(20),
    wifi_user_tag_new VARCHAR(10) CHECK(wifi_user_tag_new IN("no_wifi","Med_prob_wifi")),
    y_pred VARCHAR(2)  CHECK( y_pred IN("0","1")),
    per_difference_outgoing_calls FLOAT,
    per_difference_fb_act_hour FLOAT,
    outgoingcallsdormancydays FLOAT,
    per_difference_data_volume FLOAT,
    recharge_recency VARCHAR(2)  CHECK( recharge_recency IN("0","1")),
    bonus_frequency_last_3_months VARCHAR(2)  CHECK(bonus_frequency_last_3_months IN("0","1")),
    paidvideouserstag VARCHAR(3) CHECK(paidvideouserstag IN("No","Yes")),
    news_financial_rider_users_tag VARCHAR(3)CHECK(news_financial_rider_users_tag IN("0","1")) ,
    throughput VARCHAR(20)CHECK(throughput IN("Low","Medium","High","Very High")),
    video_user VARCHAR(20) CHECK(video_user IN("No video user","Paid_video_user","video_user"))
);''')

# Commit the changes and close the connection
conn.commit()

df.to_sql('democustomerdata', conn, if_exists='replace', index = False)

355467

In [8]:
torch.cuda.is_available()

True

In [9]:
torch.cuda.device_count()

2

In [10]:
# Fetching model

model_name = r"C:\Users\cemticsai\Documents\Projects\text2sql\SQL code\SQL project"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    # torch_dtype=torch.bfloat16,
    load_in_8bit=True,
    #load_in_4bit=True,
    device_map="auto",
    use_cache=True,
    offload_folder="offload", torch_dtype=torch.float16
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.7\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary c:\Users\cemticsai\anaconda3\envs\torchgpu\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll...


c:\Users\cemticsai\anaconda3\envs\torchgpu\lib\site-packages\bitsandbytes\cuda_setup\main.py:141: UserWarning: C:\Users\cemticsai\anaconda3\envs\torchgpu did not contain cudart64_110.dll as expected! Searching further paths...
  warn(msg)
c:\Users\cemticsai\anaconda3\envs\torchgpu\lib\site-packages\bitsandbytes\cuda_setup\main.py:141: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Users/cemticsai/anaconda3/envs/torchgpu/Library/mingw-w64/bin'), WindowsPath('C:/Users/cemticsai/anaconda3/envs/torchgpu/Library/usr/bin'), WindowsPath('C:/Users/cemticsai/anaconda3/envs/torchgpu/bin')}
  warn(msg)
Loading checkpoint shards: 100%|██████████| 4/4 [00:21<00:00,  5.48s/it]


In [11]:
def text2sql(question):
  prompt = """### Instructions:
    Your task is convert a question into a SQL query, given a Postgres database schema.

    ### Input:
    Generate a SQL query that answers the question `{question}`.
    This query will run on a database whose schema is represented in this string:

  CREATE TABLE democustomerdata (
      customer VARCHAR(5) NOT NULL,
      sitename VARCHAR(10) NOT NULL,
      segment VARCHAR(2) CHECK (segment IN ('MV', 'HV')) ,
      churn_tag VARCHAR(20) CHECK(churn_tag IN("Degrowth","Consistent")),
      voip_calls_count INT,
      total_data_volume_mb INT,
      total_revenue INT,
      voice_calls_count INT,
      segment_churn_tag VARCHAR(30) CHECK(segment_churn_tag IN("HV_Consistent_New","MV_Degrowth_New","MV_Consistent_New","HV_Degrowth_New")),
      voip_consistent_tag VARCHAR(30)CHECK (voip_consistent_tag IN("Zero","Consitent  users","in Consitent  users","recent")),
      work_pattern VARCHAR(30)CHECK( work_pattern IN("Inconsistent Worker","Consistent Workers","Home Bound","Others")),
      voip1_user_tag VARCHAR(20)CHECK(voip_user_tag  IN ("voip_user","not voip user")),
      ppsf_tag VARCHAR(10)CHECK(ppsf_tag IN("Low","Medium","High","Very High","NA")),
      work_from_home_tag VARCHAR(15) CHECK(work_from_home_tag IN ("Work from Office","Home bound","Work from Home","Other")),
      data_user_tag VARCHAR(10) CHECK(data_user_tag IN("very_high","high","low","very_low","mid")),
      voip2_user_tag VARCHAR(20) CHECK(voip_user_tag IN("No_Voip_User","Voip_User","Recent_Voip_user")),
      wifi_user_tag_new VARCHAR(10) CHECK(wifi_user_tag_new IN("no_wifi","Med_prob_wifi")),
      y_pred VARCHAR(2)  CHECK( y_pred IN("0","1")),
      per_difference_outgoing_calls INT,
      per_difference_fb_act_hour INT,
      outgoingcallsdormancydays INT,
      per_difference_data_volume INT,
      recharge_recency VARCHAR(2)  CHECK( recharge_recency IN("0","1")),
      bonus_frequency_last_3_months VARCHAR(2)  CHECK(bonus_frequency_last_3_months IN("0","1")),
      paidvideouserstag VARCHAR(3) CHECK(paidvideouserstag IN("No","Yes")),
      news_financial_rider_users_tag VARCHAR(3)CHECK(news_financial_rider_users_tag IN("0","1")) ,
      throughput VARCHAR(20)CHECK(throughput IN("Low","Medium","High","Very High")),
      video_user VARCHAR(20) CHECK(video_user IN("No video user","Paid_video_user","video_user"))
  );


    ### Response:
    Based on your instructions, here is the SQL query I have generated to answer the question `{question}`:
    ```sql
    """.format(question=question)



  ##################################

  eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]
  # excruciatingly slow on an V100 with 4bit quantization
  # takes around 1-2 minutes per query
  # on a single A100 40GB, takes ~10-20 seconds
  # torch.cuda.memory_summary(device=None, abbreviated=False)

  inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
  generated_ids = model.generate(
      **inputs,
      num_return_sequences=1,
      eos_token_id=eos_token_id,
      pad_token_id=eos_token_id,
      max_new_tokens=200,
      do_sample=False,
      num_beams=1
  )

  outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

  t= outputs[0].split("```sql")[-1].split("```")[0].split(";")[0].strip() + ";"
  print(t)


In [57]:
# Run Query here

text2sql("show me count of Paid Video customers with dormancy greater than 8")

SELECT count(*) AS count_of_paid_video_customers_with_dormancy_greater_than_8
     FROM   democustomerdata
     WHERE  democustomerdata.paidvideouserstag = 'Yes'
        AND democustomerdata.outgoingcallsdormancydays > 8;


In [59]:
# Get Data

c.execute('''SELECT COUNT(*) AS customer_count
FROM democustomerdata
WHERE video_user = 'Paid_video_user' AND outgoingcallsdormancydays > 8;
           ''')

for row in c.fetchall():
     print (row)

(10308,)
